In [ ]:
%load_ext autoreload
%autoreload 2

%cd ../

In [ ]:
import pandas as pd

from analysis.utils import (
    # hyperparameters
    get_hps_df,
    print_best_hp,
    plot_hps,

    # ETO and E2E results
    convert_to_long_df,
    get_df,
    plot_eto_vs_e2e,
    print_best_test_task_loss,
    print_eto_results,
)

## Hyperparameters

### Quantile regression

In [ ]:
df = get_hps_df(
    'out/storage_quantile/hyperparams_a{alpha:.2g}.csv',
    alphas=(0.01, 0.05, 0.1, 0.2)
)
display(df)
print_best_hp(df, by='alpha')
print_best_hp(df, by=['seed', 'alpha'])
plot_hps(df, by_alpha=True)

In [ ]:
df = get_hps_df(
    'out/storage_quantile_shuffle/hyperparams_a{alpha:.2g}.csv',
    alphas=(0.01, 0.05, 0.1, 0.2)
)
# display(df)
print_best_hp(df, by='alpha')
print_best_hp(df, by=['seed', 'alpha'])
plot_hps(df, by_alpha=True)

### Gaussian regression

In [ ]:
df = get_hps_df('out/storage_gaussian/hyperparams.csv')
print_best_hp(df)
print_best_hp(df, by='seed')
plot_hps(df, by_alpha=False, ylim=(0, 400))

In [ ]:
df = get_hps_df('out/storage_gaussian_shuffle/hyperparams.csv')
print_best_hp(df)
print_best_hp(df, by='seed')
plot_hps(df, by_alpha=False, ylim=(0, 400))

# ETO vs. E2E

## Quantile regression

In [ ]:
cols = [
    'seed',
    'train_pinball_loss', 'train_task_loss',
    'train_coverage', 'train_coverage_no_conformal',
    'test_pinball_loss', 'test_task_loss',
    'test_coverage', 'test_coverage_no_conformal'
]

### No shuffle

In [ ]:
eto_df = get_df(
    fmt_str='out/storage_quantile/eto_a{alpha:.2f}.csv',
    model='eto', cols=cols,
    lrs=None, l2regs=None,
    alphas=(0.01, 0.05, 0.1, 0.2),
)
print_eto_results(eto_df, print_best_hps=False)

In [ ]:
ptc_box_df = get_df(
    fmt_str='out/storage_ptc/ptc_box_a{alpha:.2f}.csv',
    cols=None, model='ptc_box',
    alphas=(0.01, 0.05, 0.1, 0.2),
)
ptc_box_df.groupby('alpha')[['test_task_loss', 'test_coverage']].agg(['mean', 'std'])

In [ ]:
e2e_df = get_df(
    fmt_str='out/storage_quantile/e2e_finetune_a{alpha:.2f}_lr{lr:.3g}_reg{l2reg:.3g}.csv',
    model='e2e', cols=cols + ['val_task_loss'],
    lrs=[1e-2, 1e-3, 1e-4],
    l2regs=[0, 1e-3, 1e-4],
    alphas=(0.01, 0.05, 0.1, 0.2),
)
print_best_test_task_loss(e2e_df, by='val_task_loss')

Experiment plotting using seaborn

In [ ]:
best_e2e_df = e2e_df.xs((1e-4, slice(None)), level=('lr', 'l2reg'))

In [ ]:
all_dfs = pd.concat([
    df[['test_task_loss', 'test_coverage', 'model']].reset_index()
    for df in [eto_df, ptc_box_df, best_e2e_df]
]).set_index(['model', 'alpha', 'seed'])
all_dfs.groupby('model')['test_task_loss'].count().all()

In [ ]:
import seaborn.objects as so
(
    so.Plot(data=all_dfs, x='alpha', y='test_task_loss', color='model')
    .add(so.Dot(), so.Jitter(), so.Dodge())
    .scale(x=so.Nominal([0.01, 0.05, 0.1, 0.2]))
)

In [ ]:
(
    so.Plot(data=all_dfs, x='alpha', y='test_task_loss', color='model')
    .add(so.Line(), so.Agg())
    .add(so.Band(), so.Est(errorbar='sd'))
)

In [ ]:
import seaborn as sns
sns.violinplot(data=all_dfs, x='alpha', y='test_task_loss', hue='model', cut=0)

### Shuffle

In [ ]:
eto_df = get_df(
    fmt_str='out/storage_quantile_shuffle/eto_a{alpha:.2f}.csv',
    model='eto', cols=cols,
    lrs=None, l2regs=None,
    alphas=(0.01, 0.05, 0.1, 0.2),
)
print_eto_results(eto_df, print_best_hps=False)

In [ ]:
ptc_box_df = get_df(
    fmt_str='out/storage_ptc_shuffle/ptc_box_a{alpha:.2f}.csv',
    cols=None, model='ptc_box',
    alphas=(0.01, 0.05, 0.1, 0.2),
)
ptc_box_df.groupby('alpha')[['test_task_loss', 'test_coverage']].agg(['mean', 'std'])

In [ ]:
e2e_df = get_df(
    fmt_str='out/storage_quantile_shuffle/e2e_finetune_a{alpha:.2f}_lr{lr:.3g}_reg{l2reg:.3g}.csv',
    model='e2e', cols=cols + ['val_task_loss'],
    lrs=[1e-2, 1e-3, 1e-4],
    l2regs=[0, 1e-4],
    alphas=(0.01, 0.05, 0.1, 0.2),
)
print_best_test_task_loss(e2e_df, by='val_task_loss')

In [ ]:
long_df = convert_to_long_df(pd.concat([eto_df, e2e_df]).reset_index())
plot_eto_vs_e2e(long_df, num_rows=4)

## Gaussian regression

In [ ]:
cols = [
    'seed',
    'train_nll_loss', 'train_task_loss',
    'train_coverage', 'train_coverage_no_conformal',
    'test_nll_loss', 'test_task_loss',
    'test_coverage', 'test_coverage_no_conformal'
]

### No Shuffle

In [ ]:
eto_df = get_df(
    fmt_str='out/storage_gaussian/eto_a{alpha:.2f}.csv',
    model='eto', cols=cols, lrs=None, l2regs=None,
    alphas=(0.01, 0.05, 0.1, 0.2)
)
print_eto_results(eto_df)

In [ ]:
ptc_ellipse_df = get_df(
    fmt_str='out/storage_ptc/ptc_ellipse_a{alpha:.2f}.csv',
    cols=None, model='ptc_ellipse',
    alphas=(0.01, 0.05, 0.1, 0.2),
)
ptc_ellipse_df.groupby('alpha')[['test_task_loss', 'test_coverage']].agg(['mean', 'std'])

In [ ]:
ptc_ellipse_johnstone_df = get_df(
    fmt_str='out/storage_ptc/ptc_ellipse_johnstone_a{alpha:.2f}.csv',
    cols=None, model='ptc_ellipse_johnstone',
    alphas=(0.01, 0.05, 0.1, 0.2),
)
ptc_ellipse_johnstone_df.groupby('alpha')[['test_task_loss', 'test_coverage']].agg(['mean', 'std'])

In [ ]:
e2e_df = get_df(
    fmt_str='out/storage_gaussian/e2e_finetune_a{alpha:.2f}_lr{lr:.3g}_reg{l2reg:.2g}.csv',
    model='e2e', cols=cols + ['val_task_loss'],
    lrs=[1e-3, 1e-4],
    l2regs=[1e-4],
    alphas=(0.01, 0.05, 0.1, 0.2)
)
print_best_test_task_loss(e2e_df, by='val_task_loss')

In [ ]:
long_df = convert_to_long_df(pd.concat([eto_df, e2e_df]).reset_index())
plot_eto_vs_e2e(long_df, num_rows=4)

### Shuffle

In [ ]:
eto_df = get_df(
    fmt_str='out/storage_gaussian_shuffle/eto_a{alpha:.2f}.csv',
    model='eto', cols=cols, lrs=None, l2regs=None,
    alphas=(0.01, 0.05, 0.1, 0.2)
)
print_eto_results(eto_df)

In [ ]:
ptc_ellipse_df = get_df(
    fmt_str='out/storage_ptc_shuffle/ptc_ellipse_a{alpha:.2f}.csv',
    cols=None, model='ptc_ellipse',
    alphas=(0.01, 0.05, 0.1, 0.2),
)
ptc_ellipse_df.groupby('alpha')[['test_task_loss', 'test_coverage']].agg(['mean', 'std'])

In [ ]:
ptc_ellipse_johnstone_df = get_df(
    fmt_str='out/storage_ptc_shuffle/ptc_ellipse_johnstone_a{alpha:.2f}.csv',
    cols=None, model='ptc_ellipse_johnstone',
    alphas=(0.01, 0.05, 0.1, 0.2),
)
ptc_ellipse_johnstone_df.groupby('alpha')[['test_task_loss', 'test_coverage']].agg(['mean', 'std'])

In [ ]:
e2e_df = get_df(
    fmt_str='out/storage_gaussian_shuffle/e2e_finetune_a{alpha:.2f}_lr{lr:.3g}_reg{l2reg:.2g}.csv',
    model='e2e', cols=cols + ['val_task_loss'],
    lrs=[1e-2, 1e-3, 1e-4],
    l2regs=[1e-3],
    alphas=(0.01, 0.05, 0.1, 0.2)
)
print_best_test_task_loss(e2e_df, by='val_task_loss')

In [ ]:
long_df = convert_to_long_df(pd.concat([eto_df, e2e_df]).reset_index())
plot_eto_vs_e2e(long_df, num_rows=4)

## PICNN

In [ ]:
cols = [
    'seed', 'train_task_loss', 'train_coverage',
    'test_task_loss', 'test_coverage',
]

### No Shuffle

In [ ]:
eto_df = get_df(
    fmt_str='out/storage_picnn/eto_a{alpha:.2f}_L2_d64_lr{lr:.3g}_reg{l2reg:.3g}.csv',
    model='eto', cols=cols,
    lrs=[1e-2, 1e-3, 1e-4],
    l2regs=[1e-2, 1e-3, 1e-4],
    alphas=(0.01, 0.05, 0.1, 0.2)
)
print_best_test_task_loss(eto_df, by='train_task_loss')

In [ ]:
e2e_df = get_df(
    fmt_str='out/storage_picnn/e2e_finetune_a{alpha:.2f}_L2_d64_lr{lr:.3g}_reg{l2reg:.2g}.csv',
    model='e2e', cols=cols + ['val_task_loss'],
    lrs=[1e-3, 1e-4],
    l2regs=[1e-4],
    alphas=(0.01, 0.05, 0.1, 0.2)
)
print_best_test_task_loss(e2e_df, by='val_task_loss')

### Shuffle

In [ ]:
eto_df = get_df(
    fmt_str='out/storage_picnn_zero_shuffle/eto_a{alpha:.2f}_L2_d64_lr{lr:.3g}_reg{l2reg:.3g}.csv',
    model='eto', cols=cols,
    lrs=[1e-2, 1e-3, 1e-4],
    l2regs=[1e-2, 1e-3, 1e-4],
    alphas=(0.01, 0.05, 0.1, 0.2)
)
print_best_test_task_loss(eto_df, by='train_task_loss')

In [ ]:
e2e_df = get_df(
    fmt_str='out/storage_picnn_zero_shuffle/e2e_finetune_a{alpha:.2f}_L2_d64_lr{lr:.3g}_reg{l2reg:.2g}.csv',
    model='e2e', cols=cols + ['val_task_loss'],
    lrs=[1e-3, 1e-4],
    l2regs=[1e-3],
    alphas=(0.01, 0.05, 0.1, 0.2)
)
print_best_test_task_loss(e2e_df, by='val_task_loss')

In [ ]:
long_df = convert_to_long_df(pd.concat([eto_df, e2e_df]).reset_index())
plot_eto_vs_e2e(long_df, num_rows=2)